In [70]:
import warnings
warnings.filterwarnings('ignore')

import os
import pickle

import numpy as np
import pandas as pd

from glob import glob
from pathlib import Path
from itertools import combinations

INTERVAL = 60

In [71]:
# def read_column(filename, feature):
#     print("\t\tFile: ", filename, " " * 5, end="\r")
#     value = pd.read_parquet(filename, columns=[feature])[feature].to_list()
#     column = os.path.splitext(os.path.basename(filename))[0]
#     return column, value

create src combinations of timeseries feature

In [72]:
features = ["packets", "bytes", "flows", "bytes/packets", "flows/(bytes/packets)", "nDstIP", "nSrcPort", "nDstPort"]

original_prefix = f'pch/20230329-20230331/interval_{INTERVAL}_src_feature'
# extended_prefix = ["_reconstructed_STL_trend", "_reconstructed_STL_seasonal", "_reconstructed_STL_combined", "_reconstructed_STL_detrend"]

In [73]:
for prefix in [original_prefix]:
    print("Prefix: ", prefix)
    dirname = os.path.join("timeseries", prefix)
    targetFilename = os.path.join(dirname, "pyts_dataset.npy")
    sampleFilename = os.path.splitext(targetFilename)[0] + "_sample.txt"
    featureFilename = os.path.splitext(targetFilename)[0] + "_feature.txt"
    if os.path.exists(featureFilename):
        continue

    timeseries = {}
    for feature in features:
        print("\tFeature: ", feature)
        result = {}
        for filename in glob(Path(f'{prefix}/*').__str__()):
            print("\t\tFile: ", filename, " " * 5, end="\r")
            value = pd.read_parquet(filename, columns=[feature])[feature].to_list()
            column = os.path.splitext(os.path.basename(filename))[0]
            result[column] = value
        print()
            
        timeseries[feature] = result

    os.makedirs(os.path.dirname(targetFilename), exist_ok=True)
    pyts_dataset = np.array(np.array(pd.DataFrame.from_dict(timeseries)).tolist())
    np.save(targetFilename, pyts_dataset)

    with open(os.path.join(dirname, 'timeseries_dictionary.pickle'), 'wb') as f:
        pickle.dump(timeseries, f)

    with open(sampleFilename, "w") as f:
        f.write('\n'.join(list(timeseries[features[0]].keys())))
        
    with open(featureFilename, "w") as f:
        f.write('\n'.join(features))

Prefix:  pch/20230330/interval_60_src_feature
	Feature:  packets
		File:  pch\20230330\interval_60_src_feature\99.86.178.54.parquet         
	Feature:  bytes
		File:  pch\20230330\interval_60_src_feature\99.86.178.54.parquet         
	Feature:  flows
		File:  pch\20230330\interval_60_src_feature\99.86.178.54.parquet         
	Feature:  bytes/packets
		File:  pch\20230330\interval_60_src_feature\99.86.178.54.parquet         
	Feature:  flows/(bytes/packets)
		File:  pch\20230330\interval_60_src_feature\99.86.178.54.parquet         
	Feature:  nDstIP
		File:  pch\20230330\interval_60_src_feature\99.86.178.54.parquet         
	Feature:  nSrcPort
		File:  pch\20230330\interval_60_src_feature\99.86.178.54.parquet         
	Feature:  nDstPort
		File:  pch\20230330\interval_60_src_feature\99.86.178.54.parquet         


In [74]:
for prefix in [original_prefix]:
    print("Prefix: ", prefix)
    dirname = os.path.join("timeseries", prefix)
    if "timeseries" not in locals():
        with open(os.path.join(dirname, 'timeseries_dictionary.pickle'), 'rb') as f:
            timeseries = pickle.load(f)

    targetDirname = dirname.replace("timeseries", "timeseries_feature")

    for feature in combinations(features, 2):
        print("\tFeature: ", feature)
        dirname = os.path.join(targetDirname, f"{feature[0].replace('/', '_')}-{feature[1].replace('/', '_')}")
        targetFilename = os.path.join(dirname, "pyts_dataset.npy")
        featureFilename = os.path.splitext(targetFilename)[0] + "_feature.txt"
        if os.path.exists(featureFilename):
            continue
        
        partial_timeseries = {key: value for key, value in timeseries.items() if key in feature}
        
        os.makedirs(os.path.dirname(targetFilename), exist_ok=True)
        pyts_dataset = np.array(np.array(pd.DataFrame.from_dict(partial_timeseries)).tolist())
        np.save(targetFilename, pyts_dataset)
        
        with open(featureFilename, "w") as f:
            f.write('\n'.join(feature))

Prefix:  pch/20230330/interval_60_src_feature
	Feature:  ('packets', 'bytes')
	Feature:  ('packets', 'flows')
	Feature:  ('packets', 'bytes/packets')
	Feature:  ('packets', 'flows/(bytes/packets)')
	Feature:  ('packets', 'nDstIP')
	Feature:  ('packets', 'nSrcPort')
	Feature:  ('packets', 'nDstPort')
	Feature:  ('bytes', 'flows')
	Feature:  ('bytes', 'bytes/packets')
	Feature:  ('bytes', 'flows/(bytes/packets)')
	Feature:  ('bytes', 'nDstIP')
	Feature:  ('bytes', 'nSrcPort')
	Feature:  ('bytes', 'nDstPort')
	Feature:  ('flows', 'bytes/packets')
	Feature:  ('flows', 'flows/(bytes/packets)')
	Feature:  ('flows', 'nDstIP')
	Feature:  ('flows', 'nSrcPort')
	Feature:  ('flows', 'nDstPort')
	Feature:  ('bytes/packets', 'flows/(bytes/packets)')
	Feature:  ('bytes/packets', 'nDstIP')
	Feature:  ('bytes/packets', 'nSrcPort')
	Feature:  ('bytes/packets', 'nDstPort')
	Feature:  ('flows/(bytes/packets)', 'nDstIP')
	Feature:  ('flows/(bytes/packets)', 'nSrcPort')
	Feature:  ('flows/(bytes/packets)',

create dst combinations of timeseries feature

In [75]:
features = ["packets", "bytes", "flows", "bytes/packets", "flows/(bytes/packets)", "nSrcIP", "nSrcPort", "nDstPort"]

original_prefix = f'pch/20230329-20230331/interval_{INTERVAL}_dst_feature'
# extended_prefix = ["_reconstructed_STL_trend", "_reconstructed_STL_seasonal", "_reconstructed_STL_combined", "_reconstructed_STL_detrend"]

In [76]:
for prefix in [original_prefix]:
    print("Prefix: ", prefix)
    dirname = os.path.join("timeseries", prefix)
    targetFilename = os.path.join(dirname, "pyts_dataset.npy")
    sampleFilename = os.path.splitext(targetFilename)[0] + "_sample.txt"
    featureFilename = os.path.splitext(targetFilename)[0] + "_feature.txt"
    if os.path.exists(featureFilename):
        continue

    timeseries = {}
    for feature in features:
        print("\tFeature: ", feature)
        result = {}
        for filename in glob(Path(f'{prefix}/*').__str__()):
            print("\t\tFile: ", filename, " " * 5, end="\r")
            value = pd.read_parquet(filename, columns=[feature])[feature].to_list()
            column = os.path.splitext(os.path.basename(filename))[0]
            result[column] = value
        print()
            
        timeseries[feature] = result

    os.makedirs(os.path.dirname(targetFilename), exist_ok=True)
    pyts_dataset = np.array(np.array(pd.DataFrame.from_dict(timeseries)).tolist())
    np.save(targetFilename, pyts_dataset)

    with open(os.path.join(dirname, 'timeseries_dictionary.pickle'), 'wb') as f:
        pickle.dump(timeseries, f)

    with open(sampleFilename, "w") as f:
        f.write('\n'.join(list(timeseries[features[0]].keys())))
        
    with open(featureFilename, "w") as f:
        f.write('\n'.join(features))

Prefix:  pch/20230330/interval_60_dst_feature
	Feature:  packets
		File:  pch\20230330\interval_60_dst_feature\99.86.178.54.parguet         
	Feature:  bytes
		File:  pch\20230330\interval_60_dst_feature\99.86.178.54.parguet         
	Feature:  flows
		File:  pch\20230330\interval_60_dst_feature\99.86.178.54.parguet         
	Feature:  bytes/packets
		File:  pch\20230330\interval_60_dst_feature\99.86.178.54.parguet         
	Feature:  flows/(bytes/packets)
		File:  pch\20230330\interval_60_dst_feature\99.86.178.54.parguet         
	Feature:  nSrcIP
		File:  pch\20230330\interval_60_dst_feature\99.86.178.54.parguet         
	Feature:  nSrcPort
		File:  pch\20230330\interval_60_dst_feature\99.86.178.54.parguet         
	Feature:  nDstPort
		File:  pch\20230330\interval_60_dst_feature\99.86.178.54.parguet         


In [77]:
for prefix in [original_prefix]:
    print("prefix: ", prefix)
    dirname = os.path.join("timeseries", prefix)
    with open(os.path.join(dirname, 'timeseries_dictionary.pickle'), 'rb') as f:
        timeseries = pickle.load(f)

    targetDirname = dirname.replace("timeseries", "timeseries_feature")

    for feature in combinations(features, 2):
        print("\tfeature: ", feature)
        dirname = os.path.join(targetDirname, f"{feature[0].replace('/', '_')}-{feature[1].replace('/', '_')}")
        targetFilename = os.path.join(dirname, "pyts_dataset.npy")
        featureFilename = os.path.splitext(targetFilename)[0] + "_feature.txt"
        if os.path.exists(featureFilename):
            continue
        
        partial_timeseries = {key: value for key, value in timeseries.items() if key in feature}
        
        os.makedirs(os.path.dirname(targetFilename), exist_ok=True)
        pyts_dataset = np.array(np.array(pd.DataFrame.from_dict(partial_timeseries)).tolist())
        np.save(targetFilename, pyts_dataset)

        with open(featureFilename, "w") as f:
            f.write('\n'.join(feature))

prefix:  pch/20230330/interval_60_dst_feature
	feature:  ('packets', 'bytes')
	feature:  ('packets', 'flows')
	feature:  ('packets', 'bytes/packets')
	feature:  ('packets', 'flows/(bytes/packets)')
	feature:  ('packets', 'nSrcIP')
	feature:  ('packets', 'nSrcPort')
	feature:  ('packets', 'nDstPort')
	feature:  ('bytes', 'flows')
	feature:  ('bytes', 'bytes/packets')
	feature:  ('bytes', 'flows/(bytes/packets)')
	feature:  ('bytes', 'nSrcIP')
	feature:  ('bytes', 'nSrcPort')
	feature:  ('bytes', 'nDstPort')
	feature:  ('flows', 'bytes/packets')
	feature:  ('flows', 'flows/(bytes/packets)')
	feature:  ('flows', 'nSrcIP')
	feature:  ('flows', 'nSrcPort')
	feature:  ('flows', 'nDstPort')
	feature:  ('bytes/packets', 'flows/(bytes/packets)')
	feature:  ('bytes/packets', 'nSrcIP')
	feature:  ('bytes/packets', 'nSrcPort')
	feature:  ('bytes/packets', 'nDstPort')
	feature:  ('flows/(bytes/packets)', 'nSrcIP')
	feature:  ('flows/(bytes/packets)', 'nSrcPort')
	feature:  ('flows/(bytes/packets)',

In [78]:
# np.load("reconstructed_STL_trend_ts/200702111400/pyts_dataset.npy").shape

In [79]:
# # 12 samples, 8 features, 901 timestamps
# np.array(np.array(pd.DataFrame.from_dict(timeseries)).tolist()).shape

In [80]:
# from tslearn.utils import from_pyts_dataset
# from_pyts_dataset(np.array(np.array(pd.DataFrame.from_dict(timeseries)).tolist())).shape